In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras

In [3]:
df = pd.read_csv(r'cleanedData/train_scalled.csv')
df_test = pd.read_csv(r'cleanedData/test_scalled.csv')

# SPLIT

In [4]:
# Separate the target variable from the features
target = df['Y']
features = df.drop('Y',axis=1)
# features = df[labels]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42, stratify = target)


# RANDOM FOREST

In [5]:
rf = RandomForestClassifier(n_estimators= 1000, max_depth= 4)
# Fit the classifier on the training set
rf.fit(X_train, y_train)


RandomForestClassifier(max_depth=4, n_estimators=1000)

# GRADIENT BOOST

In [6]:
# Create a Gradient Boosting Classifier model
gbc = GradientBoostingClassifier(n_estimators = 1000, max_depth=4)
# Fit the model to the data
gbc.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=4, n_estimators=1000)

# LINEAR SVC

In [7]:
# Initialize the LinearSVC model with L1 penalty and squared hinge loss
model_l1 = LinearSVC(penalty="l1", loss="squared_hinge", dual=False, tol=1e-3)
# Train the model on the selected features
model_l1.fit(X_train, y_train)


c:\Users\muham\miniconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(dual=False, penalty='l1', tol=0.001)

# NEURAL NETWORK 1

In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
y = y_train
X = X_train

model = Sequential()
model.add(Dense(256, input_dim=X.shape[1], activation='tanh'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.00001), metrics=['accuracy'])
def schedule(epoch, lr):    
    if epoch > 5:
        return lr * tf.math.exp(-0.01)
    else:
        return lr
model_learningRate_schedular = tf.keras.callbacks.LearningRateScheduler(schedule, verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=50,
    mode="auto",
    restore_best_weights=True,
)
my_callbacks = [model_learningRate_schedular, early_stopping]
# Fit the model to the data
model.fit(X, y, epochs=200, batch_size=128, validation_data=(X_test,y_test),callbacks=my_callbacks)



Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-06.
Epoch 1/200
90/90 [==============================] - 2s 7ms/step - loss: 0.6241 - accuracy: 0.7296 - val_loss: 0.5860 - val_accuracy: 0.7471 - lr: 1.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-06.
Epoch 2/200
90/90 [==============================] - 0s 5ms/step - loss: 0.5746 - accuracy: 0.7472 - val_loss: 0.5674 - val_accuracy: 0.7471 - lr: 1.0000e-05

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-06.
Epoch 3/200
90/90 [==============================] - 0s 5ms/step - loss: 0.5636 - accuracy: 0.7472 - val_loss: 0.5604 - val_accuracy: 0.7471 - lr: 1.0000e-05

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-06.
Epoch 4/200
90/90 [==============================] - 0s 5ms/step - loss: 0.5573 - accuracy: 0.7472 - val_loss: 0.5547 - val_accuracy: 0.7471 - lr: 1.0000e-05

Epoch 5: LearningRateScheduler setting learning

# KNN Classifier

In [9]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=126)
neigh.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=126)

# ENSAMBLE

In [10]:
def majority_voting(a, b, c, d, e):
    ensemble_pred = []
    t = 0
    for i in range(len(a)):
        count = {0: 0, 1: 0} 
        
        count[a[i]] += 1
        count[b[i]] += 1
        count[c[i]] += 1
        count[e[i]] += 1
        
        count[d[i][0]] += 1
        
        majority_label = max(count, key=count.get)
        ensemble_pred.append(majority_label)
        
    return ensemble_pred

In [11]:
preds1 = rf.predict(X_test)
preds2 = gbc.predict(X_test)
preds3 = model_l1.predict(X_test)
preds4 = model.predict(X_test)
preds4 = preds4 >= 0.5
preds4 = np.multiply(preds4, 1)
preds5 = neigh.predict(X_test)

154/154 [==============================] - 0s 2ms/step


In [12]:
y_pred = majority_voting(preds1, preds2, preds3, preds4, preds5)

In [13]:
# Generate a confusion matrix and calculate the metrics
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and metrics
print("Confusion matrix:")
print(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Confusion matrix:
Predicted    0     1   All
True                      
0          199  1040  1239
1           91  3569  3660
All        290  4609  4899
Accuracy: 0.7691365584813227
Precision: 0.7520612169384308
Recall: 0.7691365584813227
F1 Score: 0.7107395229109342


# Fit with remaining data and test

In [14]:
rf.fit(X_test, y_test)
gbc.fit(X_test,y_test)
model_l1.fit(X_test,y_test)
model.fit(X_test,y_test,epochs=200, batch_size=128, validation_data=(X_test,y_test),callbacks=my_callbacks)
neigh.fit(X_test,y_test)


Epoch 1: LearningRateScheduler setting learning rate to 2.3692789454798913e-06.
Epoch 1/200
17/39 [============>.................] - ETA: 0s - loss: 0.5245 - accuracy: 0.7574

c:\Users\muham\miniconda3\envs\tf\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


39/39 [==============================] - 0s 12ms/step - loss: 0.5041 - accuracy: 0.7712 - val_loss: 0.5040 - val_accuracy: 0.7714 - lr: 2.3693e-06

Epoch 2: LearningRateScheduler setting learning rate to 2.3692789454798913e-06.
Epoch 2/200
39/39 [==============================] - 0s 10ms/step - loss: 0.5040 - accuracy: 0.7708 - val_loss: 0.5039 - val_accuracy: 0.7710 - lr: 2.3693e-06

Epoch 3: LearningRateScheduler setting learning rate to 2.3692789454798913e-06.
Epoch 3/200
39/39 [==============================] - 0s 10ms/step - loss: 0.5039 - accuracy: 0.7706 - val_loss: 0.5039 - val_accuracy: 0.7710 - lr: 2.3693e-06

Epoch 4: LearningRateScheduler setting learning rate to 2.3692789454798913e-06.
Epoch 4/200
39/39 [==============================] - 0s 10ms/step - loss: 0.5039 - accuracy: 0.7712 - val_loss: 0.5038 - val_accuracy: 0.7714 - lr: 2.3693e-06

Epoch 5: LearningRateScheduler setting learning rate to 2.3692789454798913e-06.
Epoch 5/200
39/39 [==============================] -

KNeighborsClassifier(n_neighbors=126)

In [15]:
preds1 = rf.predict(X_test)
preds2 = gbc.predict(X_test)
preds3 = model_l1.predict(X_test)
preds4 = model.predict(X_test)
preds4 = preds4 >= 0.5
preds4 = np.multiply(preds4, 1)
preds5 = neigh.predict(X_test)

154/154 [==============================] - 0s 2ms/step


In [16]:
y_pred = majority_voting(preds1, preds2, preds3, preds4, preds5)

In [17]:
# Generate a confusion matrix and calculate the metrics
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the confusion matrix and metrics
print("Confusion matrix:")
print(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Confusion matrix:
Predicted    0     1   All
True                      
0          249   990  1239
1           55  3605  3660
All        304  4595  4899
Accuracy: 0.7866911614615227
Precision: 0.793281494393612
Recall: 0.7866911614615227
F1 Score: 0.7341427719588748


# Output results 

In [18]:
preds1 = rf.predict(df_test)
preds2 = gbc.predict(df_test)
preds3 = model_l1.predict(df_test)
preds4 = model.predict(df_test)
preds4 = preds4 >= 0.5
preds4 = np.multiply(preds4, 1)
preds5 = neigh.predict(df_test)

128/128 [==============================] - 0s 2ms/step


In [19]:
y_pred = majority_voting(preds1, preds2, preds3, preds4, preds5)

In [2]:
import pandas as pd

In [3]:
df_pred = pd.read_csv(r"predictions.csv")

In [5]:
# df_pred["Prediction"] = y_pred 
df_pred["Prediction"].replace({0:False,1:True},inplace=True)
df_pred["Prediction"].replace({False:1,True:0},inplace=True)

In [6]:
df_pred.to_csv("predictions.csv", index = False)